# !! Сервис отдаёт устаревшие сведения о кадастровой цене !!

Скрипт длоя получения данных из публичной кадастровой арты Росреестра.

Процеес получения сведений: загрузить файл с перечнем кадастровых номеров, запустиить первые два блок кода, забрать файл "out.xlsx"

Как оформить исходные данные:
Список кадастровых номеров должен быть в файле **in.csv**, который нужно положить файлы на левой панели.
Первая строка должна содержать две латинские буквы **cn**
Т.е.файл выглядит примерно вот так:
```
cn
26:34:80112:128
26:34:80112:127
```




In [ ]:
# Блок 1 - запустить первым
### Query to PKK for OKS

import urllib.request
import json
import csv
import time
import pandas as pd

#%% Functions

#Чтение набора участков в кадастровом квартале
def CadDistrQuery(distr, lim):
    jsn = urllib.request.urlopen("https://pkk.rosreestr.ru/api/features/1?sqo="+ distr + "&sqot=3&limit=" + str(lim)).read()
    parsed_string = json.loads(jsn)
    lplotlist = parsed_string['features']
    return lplotlist

# #Поиск ОКС в границах квартала
# def OksQueryOld(quart):
#     jsn = urllib.request.urlopen("https://pkk.rosreestr.ru/api/features/5?sqo="+ quart + "&sqot=2&").read()
#     parsed_string = json.loads(jsn)
#     lplotlist = parsed_string['features']
#     return lplotlist

#Получение сведений об объекте
def OKSQuery(oksid):
    url = 'http://rosreestr.ru/api/online/fir_object/' #урл запроса
    jsn = urllib.request.urlopen(url + oksid).read() #get server responce
    parsed_string = json.loads(jsn) #parse json to dictionary
    #Define type
    if parsed_string['parcelData']['oksFlag'] == 1:
      #filter dictionary
      returndict = {
      'Type' : 'ОКС',
      'ID': parsed_string['objectData']['id'],
      'objName' : parsed_string['objectData']['objectName'],
      'subtown' : parsed_string['objectData']['objectAddress']['locality'],
      'address' : parsed_string['objectData']['objectAddress']['mergedAddress'],
      'IDparc': parsed_string['parcelData']['id'],
      'area' : float(parsed_string['parcelData']['areaValue']),
      'areatype' : parsed_string['parcelData']['areaType'],
      'areaunit' : parsed_string['parcelData']['areaUnit'],
      'cost' : float(parsed_string['parcelData']['cadCost']),
      'buildtype' : parsed_string['parcelData']['oksType'],
      'year' : parsed_string['parcelData']['oksYearBuilt'],
      'floors' : parsed_string['parcelData']['oksFloors'],
      'isRemoved' : parsed_string['objectData']['removed']
      }
    else:   
      returndict = {
      'Type' : 'Земельный участок',
      'ID': parsed_string['objectData']['id'],
      # 'objName' : parsed_string['objectData']['objectName'],
      'subtown' : parsed_string['objectData']['objectAddress']['locality'],
      'address' : parsed_string['objectData']['objectAddress']['mergedAddress'],
      # 'IDparc': parsed_string['parcelData']['id'],
      'area' : float(parsed_string['parcelData']['areaValue']),
      'areatype' : parsed_string['parcelData']['areaType'],
      'areaunit' : parsed_string['parcelData']['areaUnit'],
      'cost' : float(parsed_string['parcelData']['cadCost']),
      'buildtype' : parsed_string['parcelData']['oksType'],
      'year' : parsed_string['parcelData']['oksYearBuilt'],
      # 'floors' : parsed_string['parcelData']['oksFloors'],
      'isRemoved' : parsed_string['objectData']['removed'],
      'catID' : parsed_string['parcelData']['categoryType'],
      'catName' : parsed_string['parcelData']['categoryTypeValue'],
      'vriCode': parsed_string['parcelData']['utilCode'],
      'vriName' : parsed_string['parcelData']['utilByDoc'],
      'vriDescr' : parsed_string['parcelData']['utilCodeDesc']
      }
    return returndict
    
def CleanCN(cnum): #function to convert cadnums to ID (without leading zeroes)
  cnum = cnum.strip().replace(' ','')
  ls = cnum.split(':')
  cnum_clear = ls[0]+':'+ls[1]+':'+ls[2].lstrip('0')+':'+ls[3].lstrip('0')
  return cnum_clear

In [ ]:
#Блок 2 - запустить после Блока 1
#Get data from in.csv (first row must be cn) list of cadnumbers and export to out.excel 
#!!!File will be RE_WRITED!!!!

cnimport = pd.read_csv('in.csv') #read input file
output = pd.DataFrame()
for i in cnimport.cn:
  s2 = CleanCN(i)
  dat = OKSQuery(s2)
  output = output.append(dat, ignore_index=True)
  time.sleep(1)
  output.to_excel('out.xlsx')  
print('Заберите файл out.xlsx в блоке слева')

Заберите файл out.xlsx в блоке слева


Блок для получения данных об одном (или нескольких, через запятую) кадастровом номере


In [ ]:
######Блок 3 - запустить после Блока 1!
#ввод нескольких кадномеров через запятую (появится окно ввода)
multi_cn = input('Введите кадастровый номер/номера')
mcnlist = multi_cn.split(',')
mDf = pd.DataFrame()
for n in enumerate(mcnlist):
  ic = CleanCN(n[1])
  try:
    mDf = mDf.append([OKSQuery(ic)], ignore_index=True)
  except:
    print('Номер',n[1],"получить не удалось")
  else:
    print (n[0]+1,'из',len(mcnlist),'загружено')
  time.sleep(1)
mDf


Введите кадастровый номер/номера26:34:010101:171
1 из 1 загружено


,Type,ID,objName,subtown,address,IDparc,area,areatype,areaunit,cost,buildtype,year,floors,isRemoved
0,ОКС,26:34:10101:171,Жилой дом,None,"г Кисловодск, ул Тюленева, д. 3",26:34:10101:171,29.8,008,055,1552233.43,building,None,1,0


In [ ]:
#Блок 4 - экспорт участков в Excel - запустить после Блока 3
mDf.to_excel('manual_out.xlsx')
print("Заберите файл manual_out.xlsx в блоке слева")

Заберите файл manual_out.xlsx в блоке слева


**Ниже - тестовый код, не вияет на функиональность, его не нужно запускать**

In [ ]:
######Блок 3 - запустить после Блока 1!
#ввод нескольких кадномеров через запятую (появится окно ввода)
multi_cn = input()
mcnlist = multi_cn.split(',')
mDf = pd.DataFrame()
for n in enumerate(mcnlist):
  ic = CleanCN(n[1])
  mDf = mDf.append([OKSQuery(ic)], ignore_index=True)
  print (n[0]+1,'из',len(mcnlist),'загружено')
  time.sleep(1)
mDf

multi_cn = input() mcnlist = multi_cn.split(',') mDf = pd.DataFrame() for n in enumerate(mcnlist):   ic = CleanCN(n[1])   mDf = mDf.append([OKSQuery(ic)], ignore_index=True)   print (n[0]+1,'из',len(mcnlist),'загружено')   time.sleep(1) mDf


IndexError: ignored

In [ ]:
single_cn = CleanCN(input())
sDf = pd.DataFrame([OKSQuery(single_cn)])
sDf


26:34:80112:127


,Type,ID,subtown,address,area,areatype,areaunit,cost,buildtype,year,isRemoved,catID,catName,vriCode,vriName,vriDescr
0,Земельный участок,26:34:80112:127,None,"г Кисловодск, ул Одесская, 40",589.0,009,055,1458841.09,None,None,0,003002000000,Земли населенных пунктов,142001000000,Под жилую застройку Индивидуальную,Для объектов жилой застройки


In [ ]:
mDf

""


In [ ]:
##test getting all plots

test2 = CadDistrQuery('26:34','')

In [ ]:
#Get data from MANUAL list of cadnumbers and export to out_man.excel 
#!!!File will be RE_WRITED!!!!

cnlist = ['26:34:80112:2648','26:34:80112:127']
output = pd.DataFrame()
for i in cnlist:
  dat = OKSQuery(i)
  output = output.append(dat, ignore_index=True)
output.to_excel('out_man.xlsx')

In [ ]:
a = input()
b = CleanCN(a)
print(b)

26:34:0080112:0127 
26:34:80112:127


In [ ]:
aaa = str('1:  2:00: 34')
aaa.strip(' ')

'1:  2:00: 34'

In [ ]:
CleanCN('1:  2:00: 34')

'1:2::34'

In [ ]:
def CleanCN2(cnum): #function to convert cadnums to ID (without leading zeroes)
  cnum = cnum.strip().replace(' ','')
  ls = cnum.split(':')
  cnum_clear = ""
  for i in ls:
    if len(i) > 1:
      print(len(i))
      cnum_clear += i.lstrip('0')
    else:
      cnum_clear += i
    cnum_clear += ':'
  # cnum_clear = ls[0]+':'+ls[1]+':'+ls[2].lstrip('0')+':'+ls[3].lstrip('0')
  return cnum_clear[:-1]
CleanCN2('1:  2:00: 34')

2
2


'1:2::34'

In [ ]:
#Поиск ОКС в границах квартала
def OksQueryOld(quart):
    jsn = urllib.request.urlopen("https://pkk.rosreestr.ru/api/features/5?sqo="+ quart + "&sqot=2&").read()
    parsed_string = json.loads(jsn)
    lplotlist = parsed_string['features']
    return lplotlist

In [ ]:
okslist = OksQueryOld('29:1:190136')

In [ ]:
for i in okslist:
  print (i['attrs']['id'])

29:1:190136:94
29:1:190136:98
29:1:190136:114
29:1:190136:251
29:1:190136:517
29:1:190136:518
29:1:0:4236
29:1:0:3905
29:1:0:3482
29:1:0:3489
29:1:0:5943
29:1:190136:505
29:1:190136:239
29:1:190136:243
